# Rede Neural Artificil


In [ ]:
import numpy
import pandas
import matplotlib

# Estrutura da Rede
pesos, camadas, etc

# 3 Funções de ativação

# 2 Funções de perda

# Algoritmo de retropropagação

# Otimização por gradiente